In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../Datos/tesis_final.csv")
df.head()

,idPaciente,Mes,TAS,Adherencia,Sexo,Edad,tas_basal
0,4026,1,119.000000,1,0,76.0,116
1,4026,2,127.000000,1,0,76.0,116
2,4026,3,140.000000,1,0,76.0,116
3,4026,4,146.712710,1,0,76.0,116
4,4026,5,177.708084,1,0,76.0,116


In [3]:
df_grouped_by_paciente = df.groupby("idPaciente")

# Crear Adherencia_Acum: variable de indice de performance de adherencia al tratamiento hasta el momento t
df["Adherencia_Acumulada"] = (
    df_grouped_by_paciente["Adherencia"]
    .expanding()
    .mean()
    .to_list()
)

# Crear covariable Adherencia_Total no dependiente del tiempo con el performance final de cada paciente
adherencia_promedio_por_paciente = (
    df_grouped_by_paciente["Adherencia"]
    .mean()
    .to_dict()
)
df["Adherencia_Total"] = df["idPaciente"].map(
    adherencia_promedio_por_paciente
)

# Crear Adherencia Perfecta (1 si adhiere todos los meses, 0 en otro caso)
df["Adherencia_Perfecta"] = (df["Adherencia_Total"] == 1).astype(int)

# Crear TAS_Media_Acum: variable de TAS media hasta el momento t
df["TAS_Media_Acumulada"] = (
    df_grouped_by_paciente["TAS"]
    .expanding()
    .mean()
    .to_list()
)

In [4]:
df["Adherencia_lag1"] = (
    df_grouped_by_paciente["Adherencia"]
    .shift(1)
    .fillna(0)
)

df["TAS_lag1"] = (
    df_grouped_by_paciente["TAS"]
    .shift(1)
    .fillna(df["tas_basal"])
)

df["Adherencia_Acumulada_lag2"] = (
    df_grouped_by_paciente["Adherencia_Acumulada"]
    .shift(2)
    .fillna(0)
)

df["TAS_Media_Acumulada_lag2"] = (
    df_grouped_by_paciente["TAS_Media_Acumulada"]
    .shift(2)
    .fillna(df["tas_basal"])
)

In [5]:
df.to_csv("../Datos/tesis_final_preprocesado.csv", index=False)